In [1]:
#import all packages required
import pandas as pd
import numpy as np
import requests
import json
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from bs4 import BeautifulSoup

print('library imported successfully')

library imported successfully


In [2]:
#Actually I used Spyder for the coding writing....
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
output = requests.get(url)
data_soup = BeautifulSoup(output.content, 'html.parser')
table = data_soup.find('table')
fields = table.find_all('td')

a = len(fields)
print(a)

postcode = []
borough = []
nebod = []

for i in range(0, a, 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    nebod.append(fields[i+2].text.strip())
        
loc_data = pd.DataFrame(data=[postcode, borough, nebod]).T
loc_data.columns = ['Postcode', 'Borough', 'Neighbourhood']
loc_data.head()

864


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
loc_data['Borough'].replace('Not assigned', np.nan, inplace=True)
loc_data.dropna(subset=['Borough'], inplace=True)

loc_data.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [5]:
loc_data2 = loc_data.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
loc_data2['Neighbourhood'].replace('Not assigned', "Queen's Park", inplace=True)
loc_data2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
geo = pd.read_csv('http://cocl.us/Geospatial_data')
geo.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
geo.columns = ['Postcode', 'Latitude', 'Longitude']
data_clean = pd.merge(loc_data2, geo, how='inner', on=['Postcode'])
data_gtg = data_clean[['Borough', 'Neighbourhood', 'Postcode', 'Latitude', 'Longitude']].copy()
data_gtg.head()

,Borough,Neighbourhood,Postcode,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [10]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="lol")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are ' + str(latitude) + ' and ' + str(longitude))

The coordinates ofToronto are 43.653963 and -79.387207


In [18]:
# create map of Toronto, Canada using latitude and longitude values
map_trt = folium.Map(location=[latitude, longitude], zoom_start= 9.5 )

# add markers to map
for lat, long, borough, neighborhood in zip(data_gtg['Latitude'], data_gtg['Longitude'], data_gtg['Borough'], data_gtg['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_trt)  
    
map_trt